In [1]:
import os

In [2]:
os.chdir("../")

In [3]:
%pwd


'D:\\Projects\\Stock_Market_Predictor_ETE'

In [4]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path

In [5]:
from src.stock_predictor.constants import *
from src.stock_predictor.utils.common import read_yaml, create_directories

In [6]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation

        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
        )

        return data_transformation_config

In [7]:
import os
from src.stock_predictor import logger
from src.stock_predictor.entity.config_entity import DataTransformationConfig
from sklearn.model_selection import train_test_split
import pandas as pd

In [8]:
class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config
  

    def transform_data(self):
        data = pd.read_csv(self.config.data_path)

        try:
            # Perform data transformation here
            del data["Dividends"]
            del data["Stock Splits"]

            data["Tomorrow"] = data["Close"].shift(-1)
            data["Target"] = (data["Tomorrow"] > data["Close"]).astype(int)
            data = data.loc["1990-01-01": ].copy()
            data.dropna(inplace=True)
            data.to_csv(self.config.data_path, index=False)
        except Exception as e:
            logger.error(f"Error in data transformation: {e}")
            raise e
    
    def train_test_spliting(self):
        data = pd.read_csv(self.config.data_path)

        # Split the data into training and test sets. (0.75, 0.25) split.
        train, test = train_test_split(data)

        train.to_csv(os.path.join(self.config.root_dir, "train.csv"), index=False)
        test.to_csv(os.path.join(self.config.root_dir, "test.csv"), index=False)

        logger.info("Splitted data into training and test sets")
        logger.info(train.shape)
        logger.info(test.shape)

        print(train.shape)
        print(test.shape)
        

In [9]:
        try:
            with open(Path("artifacts/data_validation/status.txt"), "r") as f:
                status = f.read().split(" ")[-1]
                print(status)

            if status == "True":
                config = ConfigurationManager()
                data_transformation_config = config.get_data_transformation_config()
                data_transformation = DataTransformation(config=data_transformation_config)
                data_transformation.transform_data()
                # data_transformation.train_test_spliting()
            else:
                raise Exception("You data schema is not valid")

        except Exception as e:
            print(e)

True
[2024-05-20 20:53:12,150: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-05-20 20:53:12,156: INFO: common: yaml file: params.yaml loaded successfully]
[2024-05-20 20:53:12,165: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-05-20 20:53:12,166: INFO: common: created directory at: artifacts]
[2024-05-20 20:53:12,167: INFO: common: created directory at: artifacts/data_transformation]
